In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
#Use koi_disposition for the y values

y = df['koi_disposition']
X = df.drop(columns=["koi_disposition"], axis=1)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.6 )

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(y_train) 

y_train_encoded = encoder.transform(y_train)
y_test_encoded = encoder.transform(y_test)

In [6]:
#One-hot encoding
from keras.utils import to_categorical


y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)
y_train_categorical.shape


Using TensorFlow backend.


(4194, 3)

In [7]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape

(4194, 40)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units = 40, activation='relu', input_dim=40))
deep_model.add(Dense(units = 40, activation='relu'))
deep_model.add(Dense(units = 3, activation='softmax'))


In [9]:
deep_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40)                1640      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 123       
Total params: 3,403
Trainable params: 3,403
Non-trainable params: 0
_________________________________________________________________


In [10]:
deep_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 4194 samples
Epoch 1/100
4194/4194 - 1s - loss: 0.7394 - accuracy: 0.6249
Epoch 2/100
4194/4194 - 0s - loss: 0.4025 - accuracy: 0.7992
Epoch 3/100
4194/4194 - 0s - loss: 0.3740 - accuracy: 0.7985
Epoch 4/100
4194/4194 - 0s - loss: 0.3563 - accuracy: 0.8264
Epoch 5/100
4194/4194 - 0s - loss: 0.3565 - accuracy: 0.8114
Epoch 6/100
4194/4194 - 0s - loss: 0.3391 - accuracy: 0.8348
Epoch 7/100
4194/4194 - 0s - loss: 0.3386 - accuracy: 0.8422
Epoch 8/100
4194/4194 - 0s - loss: 0.3276 - accuracy: 0.8429
Epoch 9/100
4194/4194 - 0s - loss: 0.3236 - accuracy: 0.8448
Epoch 10/100
4194/4194 - 0s - loss: 0.3175 - accuracy: 0.8479
Epoch 11/100
4194/4194 - 0s - loss: 0.3135 - accuracy: 0.8553
Epoch 12/100
4194/4194 - 0s - loss: 0.3097 - accuracy: 0.8524
Epoch 13/100
4194/4194 - 0s - loss: 0.3081 - accuracy: 0.8612
Epoch 14/100
4194/4194 - 0s - loss: 0.3077 - accuracy: 0.8562
Epoch 15/100
4194/4194 - 0s - loss: 0.3025 - accuracy: 0.8600
Epoch 16/100
4194/4194 - 0s - loss: 0.3066 - accuracy: 0.

In [12]:
model_loss, model_accuracy = deep_model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2797/2797 - 0s - loss: 0.2601 - accuracy: 0.8935
Loss: 0.26006367623614207, Accuracy: 0.893457293510437


In [13]:
predictions = deep_model.predict_classes(X_test_scaled)
#prediction_labels = encoder.inverse_transform(predictions)

predictions_df=pd.DataFrame({"Actual": y_test_encoded, "Prediction": predictions })
results=predictions_df.apply(pd.Series.value_counts)
code = ['False Positive', 'Confirmed', 'Candidate',] 
results['Code']=code

results

,Actual,Prediction,Code
0,672,718,False Positive
1,764,708,Confirmed
2,1361,1371,Candidate


In [15]:
deep_model.save("nn_model.h5")

In [16]:
import joblib
filename = 'nn_model.sav'
joblib.dump(deep_model, filename)
#getting error: can't pickle _thread.RLock objects

TypeError: can't pickle _thread.RLock objects